# **Extract Transform Load (ETL) Lab**


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Set Paths


In [3]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [4]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [18]:
df_csv= extract_from_csv('source3.csv')
df_csv

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


### JSON Extract Function


In [5]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [17]:
df_json = extract_from_json('source3.json')
df_json

,name,height,weight
0,jack,68.70,123.30
1,tom,69.80,141.49
2,tracy,70.01,136.46
3,john,67.90,112.37


### XML Extract Function


In [19]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

In [20]:
df_xml = extract_from_xml('source2.xml')
df_xml

,name,height,weight
0,simon,67.90,112.37
1,jacob,66.78,120.67
2,cindy,66.49,127.45
3,ivan,67.62,114.14


### Extract Function


In [7]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [21]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [22]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [23]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [24]:
log("ETL Job Started")

In [25]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,jack,68.70,123.30
6,tom,69.80,141.49
7,tracy,70.01,136.46
8,john,67.90,112.37
9,simon,67.90,112.37


In [26]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,jack,1.74,55.93
6,tom,1.77,64.18
7,tracy,1.78,61.90
8,john,1.72,50.97
9,simon,1.72,50.97


In [27]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [28]:
log("ETL Job Ended")